# Count the scaler

From *2021-03 APS Python Training for Bluesky Data Acquisition*.

**Objective**

In this notebook, we show how to count the scaler:

* on the command line
* using ophyd
* using the bluesky RunEngine

We also show how to:

* set the counting time
* choose certain channels for data acquisition

## Start the `instrument` package

In [1]:
from instrument.collection import *

I Wed-21:21:59 - ############################################################ startup
I Wed-21:21:59 - logging started
I Wed-21:21:59 - logging level = 10
I Wed-21:21:59 - /home/beams1/JEMIAN/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/collection.py
I Wed-21:21:59 - /home/beams1/JEMIAN/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/mpl/notebook.py


Activating auto-logging. Current session state plus future input saved.
Filename       : /home/beams1/JEMIAN/Documents/projects/BCDA-APS/bluesky_instrument_training/.logs/ipython_console.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


I Wed-21:22:01 - #### Bluesky Framework ####
I Wed-21:22:01 - /home/beams1/JEMIAN/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/framework/check_python.py
I Wed-21:22:01 - /home/beams1/JEMIAN/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/framework/check_bluesky.py
I Wed-21:22:04 - /home/beams1/JEMIAN/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/framework/initialize.py
I Wed-21:22:10 - using databroker catalog 'class_2021_03'
I Wed-21:22:10 - /home/beams1/JEMIAN/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/framework/metadata.py
I Wed-21:22:10 - #### Devices ####
I Wed-21:22:10 - /home/beams1/JEMIAN/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/devices/area_detector.py
I Wed-21:22:10 - /home/beams1/JEMIAN/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/devices/calculation_records.py
I Wed-21:22:13 - /home/beams1/JEMIAN/Documents/projects/BCDA-APS/bluesky_instrument

# A closer look at the EPICS scaler

A scaler is a device that counts digital pulses from a pulse detector such
as a scintillation counter or from photodiodes or ionization chamber with
pulse chain electronics.  Scalers have
many channels, some of which might have no associated detector.  Our scaler
(`scaler1`) is a simulated device that records a random number of pulses in 
each channel.  We are only interested in the channels that have names provided
by users in the GUI screens.  In this control screen for our scaler, only a few of the
channels are named:

![`scaler` GUI](resources/scaler.png "`scaler1` GUI")

Let's configure `scaler1` to report only the `diode` and `I0` channels (plus the count time channel which will *always* be included).  Keep in mind that the argument to this function is a Python list, so the channel names must be enclosed with `[]`.  The function does not return a result.  If something *is* printed, there is an error to be fixed.

In [2]:
scaler1.select_channels(["diode", "I0"])

The _easiest_ way to count the `scaler` object is to use the [`%ct`](https://blueskyproject.io/bluesky/magics.html?highlight=label#taking-a-reading-using-ct-post-v1-3-0) bluesky magic command, which counts all objects with the `detectors` label.

Note that the various magic commands are only available from the command line, not for use in a bluesky plan function.

In [3]:
%ct

[This data will not be saved. Use the RunEngine to collect data.]
noisy                          0.0
I0                             5.0
diode                          4.0
scaler1_time                   1.1


Compare with the reading when _all_ channels are selected.

NOTE:  To report _all_ named channels, call `.select_channels()` with empty parentheses.

In [4]:
scaler1.select_channels()
%ct

[This data will not be saved. Use the RunEngine to collect data.]
noisy                          0.0
timebase                       11000000.0
I0                             5.0
scint                          7.0
diode                          3.0
scaler1_time                   1.1


Now, select just the two channels again before continuing:

In [5]:
scaler1.select_channels(["diode", "I0"])

As noted before, the `%ct` command is only available from the command shell.

### use ophyd to count the scaler

We should learn how to use the underlying Python code to do the same steps.

The first step is to use pure ophyd methods to count and report, then use a bluesky plan to do the same thing.  The ophyd methods are `trigger`, `wait`, and `read`.  The `trigger` and `wait` methods can be chained together:

In [6]:
scaler1.trigger().wait()

Technically, we should `stage` and `unstage` the object.  **We'll use staging to control the count time of the scaler.**

The ophyd [`.stage()` method](https://nsls-ii.github.io/ophyd/generated/ophyd.device.BlueskyInterface.stage.html?highlight=stage#ophyd.device.BlueskyInterface.stage) prepares the object for its `.trigger()` method, while the `.unstage()` method returns the object's settings to the previous state before the `.stage()` method was called.

In [7]:
scaler1.stage()
scaler1.trigger().wait()
scaler1.unstage()

[Channels(prefix='gp:scaler1', name='scaler1_channels', parent='scaler1', read_attrs=['chan02', 'chan02.s', 'chan04', 'chan04.s'], configuration_attrs=['chan02', 'chan02.chname', 'chan02.preset', 'chan02.gate', 'chan04', 'chan04.chname', 'chan04.preset', 'chan04.gate']),
 ScalerCH(prefix='gp:scaler1', name='scaler1', read_attrs=['channels', 'channels.chan02', 'channels.chan02.s', 'channels.chan04', 'channels.chan04.s', 'time'], configuration_attrs=['channels', 'channels.chan02', 'channels.chan02.chname', 'channels.chan02.preset', 'channels.chan02.gate', 'channels.chan04', 'channels.chan04.chname', 'channels.chan04.preset', 'channels.chan04.gate', 'count_mode', 'delay', 'auto_count_delay', 'freq', 'preset_time', 'auto_count_time', 'egu'])]

Let's find out what happens when `scaler1` is staged.  That's controlled by the contents of a Python dictionary `.stage_sigs`:

In [8]:
scaler1.stage_sigs

OrderedDict()

It's empty, so nothing has been preconfigured for us.  Let's make sure that we get to pick the *counting time* (the time to accumulate pulses in the various channels), say 2.0 seconds, when we count here.

In [9]:
scaler1.stage_sigs["preset_time"] = 2
scaler1.stage_sigs

OrderedDict([('preset_time', 2)])

Show the counting time *before* we count, then `stage`, `trigger`, `wait`, `read`, `unstage`, then finally show the counting time  *after* we count:

In [10]:
print(f"Scaler configured to count for {scaler1.preset_time.get()}s")
scaler1.stage()
scaler1.trigger().wait()
print(scaler1.read())
scaler1.unstage()
print(f"Scaler configured to count for {scaler1.preset_time.get()}s")

Scaler configured to count for 1.0s
OrderedDict([('I0', {'value': 11.0, 'timestamp': 1614223342.211066}), ('diode', {'value': 9.0, 'timestamp': 1614223342.211066}), ('scaler1_time', {'value': 2.1, 'timestamp': 1614223339.960063})])
Scaler configured to count for 1.0s


The report from `.read()` includes both values and timestamps (in seconds since the Python [time](https://docs.python.org/3/library/time.html) epoch, UTC).  The structure is a Python dictionary.  This is the low-level method used to collect readings from any ophyd device.  We had to `print()` this since the return result from a command within a sequence is not returned at the end of the sequence, just the return result of the *final* command in the sequence.

See that the scaler counted for 2.1 seconds (a small bug in the scaler simulator it seems, always adds .1 to the count time!).  But before staging, the scaler was configured for 1.0 seconds, and after unstaging, the scaler returned to that value.

**That's how to control the counting time *for a scaler*.**  (Area detectors use different terms.  More on that later.)

<details>
    <summary>about <tt>scaler1_time</tt></summary>

     Of note is the key `scaler1_time` which is the name of the ophyd symbol `scaler1.time` as returned by `scaler1.time.name`:

        In [21]: scaler1.time.name
        Out [21]: 'scaler1_time'

</details>

### use bluesky (the package) to count the scaler

Now, use the bluesky [RunEngine](https://blueskyproject.io/bluesky/generated/bluesky.run_engine.RunEngine.html?highlight=runengine#bluesky.run_engine.RunEngine) (`RE`) to count `scaler1`.  We'll use the bluesky plan ([`bp`](https://blueskyproject.io/bluesky/plans.html?highlight=count#pre-assembled-plans)) called [`count()`](https://blueskyproject.io/bluesky/generated/bluesky.plans.count.html#bluesky.plans.count).  To be consistent with the result returned from `%ct`, we'll include the `noisy` detector.

In [11]:
RE(bp.count([scaler1,noisy]))



Transient Scan ID: 42810     Time: 2021-02-24 21:22:22
Persistent Unique Scan ID: '90a47616-17dd-4232-84b6-766c323459c9'
New stream: 'baseline'
New stream: 'primary'
+-----------+------------+------------+------------+------------+
|   seq_num |       time |         I0 |      diode |      noisy |
+-----------+------------+------------+------------+------------+
|         1 | 21:22:24.8 |         10 |          8 |    0.00000 |
+-----------+------------+------------+------------+------------+
generator count ['90a47616'] (scan num: 42810)


('90a47616-17dd-4232-84b6-766c323459c9',)

There are many ways to view data from bluesky runs.  The most recent run can be picked from the database using the Python reference for *the last item in the list*: `[-1]`.  We need the `v2` version of the database to access the `.primary` data stream.

We'll pick one simple to view the run data, as an [xarray](http://xarray.pydata.org/) table since it is easy to display such structured content in a Jupyter notebook.  (We could just as easily have displayed the data as a [dask](https://dask.org/) table by replacing `.read()` with `.to_dask()`.)

In [12]:
db.v2[-1].primary.read()

<xarray.Dataset>
Dimensions:       (time: 1)
Coordinates:
  * time          (time) float64 1.614e+09
Data variables:
    I0            (time) float64 10.0
    diode         (time) float64 8.0
    scaler1_time  (time) float64 2.1
    noisy         (time) float64 0.0

As a last action in this section, use the [`listruns()`](https://apstools.readthedocs.io/en/latest/source/_utils.html?highlight=listobjects#apstools.utils.listruns) command from *apstools* to show the (default: 20) most recent runs in the database.  The table shows a short version of the run's unique identifier (`short_uid`), and other more obvious columns of information, truncated to avoid lengthy output.  The name of the [databroker *catalog*](https://blueskyproject.io/databroker/index.html) (`class_2021_03`) is shown before the table.

In [13]:
listruns()

catalog name: class_2021_03
========= ========================== ======= ======= ========================================
short_uid date/time                  exit    scan_id command                                 
========= ========================== ======= ======= ========================================
90a4761   2021-02-24 21:22:22.251613 success 42810   count(detectors=['scaler1', 'noisy'] ...
dff821e   2021-02-24 19:55:30.877637 success 42809   count(detectors=['temperature'], num=50)
54623bd   2021-02-24 19:55:10.010586 success 42808   count(detectors=['temperature'], num=10)
bd1b5a0   2021-02-24 19:55:07.992559 success 42807   count(detectors=['temperature'], num=10)
188b6b4   2021-02-24 19:55:07.421838 success 42806   count(detectors=['temperature'], num=1) 
ea64830   2021-02-24 19:54:38.117091 success 42805   count(detectors=['temperature'], num=1) 
========= ========================== ======= ======= ========================================

